### CS/ECE/ISyE 524 &mdash; Introduction to Optimization &mdash; Spring 2023 ###

### Final Course Project: Due 5/5/23

# Project title goes here #

#### Lauren Ciha lciha@wisc.edu

*Disclaimer: I was not paid by Todoist to do my project on their app or to market their app in any way. I chose to do my project on the Todoist app and task dependency because I wanted to hack my to-do list, which is currently in Todoist. Many of the conclusions here transfer to other productivity apps as well.*

*****

### Table of Contents

1. [Introduction](#1.-Introduction)
1. [Mathematical Model](#2.-Mathematical-model)
1. [Solution](#3.-Solution)
1. [Results and Discussion](#4.-Results-and-discussion)
  1. [Optional Subsection](#4.A.-Feel-free-to-add-subsections)
1. [Conclusion](#5.-Conclusion)

## 1. Introduction ##

To-do lists are great for keeping track of what to do. But they rarely give you a place to start.

I use the Todoist app (put a link in here) to keep track of what I need to do. The app allows you to sort tasks into projects and further into sections. From there, you can set deadlines, add tags to sort tasks, and rate their priority.

Here's an example of some of my weekly tasks this semester:

![todolist.png](todolist.png)

You can see that I have laid out my tasks, put them in a section, and added tags estimating the amount of time it will take me to complete each task. One of these tasks has a subtask.

![subtask.png](subtask.png)

For this assignment, I prefer to print out the worksheet, fill it out, and then upload it as a pdf. In order to do my homework assignment, then, I need to *first* print out the worksheet. Normally, a subtask indicates part of a task that is included in the main task, but in this case, I'm using a subtask to express that a task depends on another, in other words, that I need to finish the subtask before marking the main task as complete.

Currently, Todoist doesn't have a way to label a task as dependent on another. In addition to subtasks, I sometimes express dependency with a '@waiting' tag and then write what I'm waiting for in the task description.

![waiting.png](waiting.png)

Although task dependency may not appear in the Todoist user interface, it's a well-known linear program. (we can define linear program here if needed). There are two models or frameworks we can use to minimize the time it takes to complete tasks when some tasks depend on others. The first is called multi-planning problem and the other is longest-path. With these two models, we can determine what order I should complete my tasks in to minimize the time it takes to do everything. 

**In this project, I'm going to model tasks with dependencies from my own to-do list. From there, I plan to use these models as a starting point for suggesting how to incorporate a task dependency feature into the Todoist app.**




The first few sentences should give a quick overview of the entire project. Then, elaborate with a description of the problem that will be solved, a brief history (with [citations](https://en.wikipedia.org/wiki/Citation)) of how the problem came about, why it's important/interesting, and any other interesting facts you'd like to talk about. You should address and explain where the problem data is coming from (research? the internet? synthetically generated?) Also give an outline of the rest of the report.

This section should be 300-600 words long, and **should be accessible to a general audience** (don't assume your reader has taken the class!). Feel free to include images if you think it'll be helpful:

For more help on using Markdown, see [this reference](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet).


## 2. Mathematical models ##

### Set up for getting into the model: what qualities does the real-world problem have?

Let's take a look at that to-do list again:

![todolist.png](todolist.png)

Here's what we know about our problem:
* We want to minimize the amount of time it's going to take to do the tasks
* We have tasks, their estimated time to complete, and travel time?
* Some tasks need to be done before other tasks
* What we're looking for in particular is the sequence of tasks, rather than the time to complete these tasks

Man, I hope my problem isn't trivial

### Modeling as a scheduling (multi-period planning problem)
* what is a multi-period planning problem?
* what are the assumptions?
* what does the graph for this problem look like?
* decision variables
* constraints
* objective

### Modeling as a longest-path problem (network flow)
* what is a longest-path planning problem?
* what are the assumptions?
* what does the graph for this problem look like?
* decision variables
* constraints
* objective

### Why look at both? Comparison between the two

Here is an example of an equation:

$$
\begin{bmatrix}
  1 & 2 \\
  3 & 4
\end{bmatrix}
\begin{bmatrix} x \\ y \end{bmatrix} =
\begin{bmatrix} 5 \\ 6 \end{bmatrix}
$$

And here is an example of an optimization problem in standard form:

$$
\begin{aligned}
\underset{x \in \mathbb{R^n}}{\text{maximize}}\qquad& f_0(x) \\
\text{subject to:}\qquad& f_i(x) \le 0 && i=1,\dots,m\\
& h_j(x) = 0 && j=1,\dots,r
\end{aligned}
$$

For some quick tips on using $\LaTeX$, see [this cheat sheet](http://users.dickinson.edu/~richesod/latex/latexcheatsheet.pdf).

## Formulation 2: 

### What's new?

### Multi-period planning

### Longest path problem

## Formulation 3:

### What's new?

### Multi-period planning

### Longest path problem

## 3. Solution ##

Here, you should code up your model in Julia + JuMP and solve it. Your code should be clean, easy to read, well annotated and commented, and it should compile! You are not allowed to use other programming languages or DCP packages such as `convex.jl`. **I will be running your code**. I suggest having multiple code blocks separated by text blocks that explain the various parts of your solution. You may also solve several versions of your problem with different models/assumptions.

It's fine to call external packages such as `Gurobi`, but try to minimize the use of exotic libraries.

In [1]:
# this is a code block
using JuMP, Clp
m=Model(Clp.Optimizer)

things = [:horses, :donkeys, :goats]  # these are the things 
@variable(m, x[things] >= 0)          # the quantities of each of the things (can't be negative)
@constraint(m, sum(x) <= 10)          # we can't have any more than 10 things total
@objective(m, Max, x[:horses])        # we want to maximize the number of horses
optimize!(m)

for i in things
    println("The total number of ", i, " is: ", value(x[i]))     # print result
end

The total number of horses is: 10.0
The total number of donkeys is: 0.0
The total number of goats is: 0.0
Coin0506I Presolve 0 (-1) rows, 0 (-3) columns and 0 (-3) elements
Clp3002W Empty problem - 0 rows, 0 columns and 0 elements
Clp0000I Optimal - objective value 10
Coin0511I After Postsolve, objective 10, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 10 - 0 iterations time 0.002, Presolve 0.00


Remember to make sure your code compiles! I will be running your code!

## 4. Results and discussion ##

Here, you display and discuss the results. Show figures, plots, images, trade-off curves, or whatever else you can think of to best illustrate your results. The discussion should explain what the results mean, and how to interpret them. You should also explain the limitations of your approach/model and how sensitive your results are to the assumptions you made.

Use plots (see `PyPlot` examples from class), or you can display results in a table like this:

| Tables        | Are           | Cool  |
| ------------- |:-------------:| -----:|
| col 3 is      | right-aligned |\$1600 |
| col 2 is      | centered      |  \$12 |
| zebra stripes | are neat      |   \$1 |

### 4.A. Feel free to add subsections

#### 4.A.a. or subsubsections

## 5. Conclusion ##

Summarize your findings and your results, and talk about at least one possible future direction; something that might be interesting to pursue as a follow-up to your project.

## Citations

### Modeling, writing, etc.

### formulating the project itself:
